Importações

In [2]:
import time
import scipy.linalg as sp
import numpy as np
import sympy as sym


## ITEM 1

In [83]:
# Test cell
Fx = [-x for x in calcF(F,x)]
Fx
Jx = calcJ(J(F), x)
np.array(Jx, np.int64)
y = sp.solve(np.array(Jx), np.array(Fx))


ValueError: object arrays are not supported

In [64]:
# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
f1 = x1*x2 +x1 -3*x5
f2 = 2*x1*x2 +x1 +3*R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 -R[0]*x5
f3 = 2*x2*(x3**2) +R[-4]*x2*x3 + 2*R[-6]*(x3**2) +R[-5]*x3 -8*x5
f4 = R[-2]*x2*x4 +2*(x4**2) -4*R[0]*x5
f5 = x1*x2 +x1 +R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 +R[-6]*(x3**2) +R[-5]*x3 +(x4**2) -1
    
F = [f1,f2,f3,f4,f5]

## calculate values of F under a vector x
def calcF(F, x):
    Fx = []
    for f in F:
        Fx.append(f.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
    return Fx

## define jacobian matrix
def J(F):
    Jmatrix = []
    for f in F:
        Jmatrix.append([f.diff(x1), f.diff(x2), f.diff(x3), f.diff(x4), f.diff(x5)])
        
    return  Jmatrix    


## calculate values of J under a vector x
def calcJ(J, x):
    Jx = []
    i = 0
    for linha in J:
        Jx.append([])
        for j in linha:
            Jx[i].append(j.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
        i += 1
        
        
    return Jx



### Método de Newton

In [97]:
# define method
def methodNewton(n, x, tolerance, maxIterations, F):
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    k = 1
    while k <= maxIterations:
        print("1 Iteração x=", x, end=" ")
        Fx = [-x for x in calcF(F,x)]
        Jx = calcJ(J(F), x)
        y = np.linalg.solve(np.array(Jx, np.int32), np.array(Fx, np.int32))
        x = x + y
        norma_y = np.linalg.norm(y) 
        if norma_y < tolerance:
            output = x
            break
        else:
            print("Norma: ", norma_y)
            k += 1
        
    return output

In [98]:
# solve with Newton's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = .0005 # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

methodNewton(n, Xo, tolerance, maxIterations, F)


1 Iteração x= [10 10 10 10 10] Norma:  16.2884636466769
1 Iteração x= [-0.51630247 10.55526018  4.71894356  4.93275821 -0.04224179] Norma:  3.593595179098674
1 Iteração x= [-0.05295344  9.20862442  2.64226144  2.36913665 -0.00996203] Norma:  2.3657508181359717
1 Iteração x= [-0.05308498  7.50923389  1.54214598  1.14496571 -0.01040049] Norma:  14.684851131423077
1 Iteração x= [-4.61405374e-02  2.20462709e+01 -4.57854022e-01  5.80150895e-01
  8.11802866e-03] 

LinAlgError: Singular matrix

### Método de Broyden

In [ ]:
Po = x # initial approximation
tolerance = .0005 # arbitrary
maxIterations = 100000 # so it doesnt loop forever in case of failure


def methodBroyden(n, x, tolerance, maxIterations): # n is the number of equations and unknowns
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    Ao = J(x)
    v = F(x)
    A = np.transpose(Ao) #use gaussian elimination
    s = -A * v
    x = x + s
    k = 2
    
    while k <= maxIterations:
        w = v
        v = F(x)
        y = v - w
        z = -A* y
        p = -np.transpose(s) * z
        ut = np.transpose(np.transpose(s) * A)
        A = A + (1/p)*(s+z)*ut
        s = -A * v
        x = x + s
        if np.linalg.norm(s) < tolerance:
            output = x
            break
        
        k += 1
        
    return output
             
    

### Comparação dos resultados

## ITEM 2

### Método de Newton

In [ ]:
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = .0005 # chosen
maxIterations = 100000 # so it doesnt loop forever in case of failure

def f(x):
    return


def methodNewton(p0, p1, tolerance, maxIterations):
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    i = 1
    q0 = f(p0)
    q1 = f(p1)
    while i <= maxIterations:
        p = p1 - q1*((p1-p0)/(q1-q0))
       
        if tolerance < p - p1 < tolerance:
            output = p
            break
        else:
            i += 1
            p0 = p1
            q0 = q1
            p1 = p
            q1 = f(p)
        
    return output

### Método de Broyden

In [ ]:
Po = x # initial approximation
tolerance = .0005 # arbitrary
maxIterations = 100000 # so it doesnt loop forever in case of failure
def f(x):
    return
def df(x):
    return
def J(x):
    result = df(x)
    return result
def F(x):
    return


def methodBroyden(n, x, tolerance, maxIterations): # n is the number of equations and unknowns
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    Ao = J(x)
    v = F(x)
    A = np.transpose(Ao) #use gaussian elimination
    s = -A * v
    x = x + s
    k = 2
    
    while k <= maxIterations:
        w = v
        v = F(x)
        y = v - w
        z = -A* y
        p = -np.transpose(s) * z
        ut = np.transpose(np.transpose(s) * A)
        A = A + (1/p)*(s+z)*ut
        s = -A * v
        x = x + s
        if np.linalg.norm(s) < tolerance:
            output = x
            break
        
        k += 1
        
    return output
          

### Comparação dos resultados

## ITEM 3

#### Bibliografia e referências: 

R.L. Burden & J.D. Faires, Numerical Analysis, Capítulos 2 e 10